# Лаб2. Content-based рекомендательная система образовательных курсов – Spark Dataframes
Задание: https://github.com/newprolab/sber-spark-ds-18/blob/main/labs/lab02.md

### Импорты и настройка окружения

In [ ]:
import findspark
findspark.init('/opt/spark-3.4.3/')

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql import functions as F
from pyspark.sql.types import *
# from pyspark import Row

import re

conf = SparkConf()
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .appName('max_burdasov_lab2')
    .getOrCreate()
)

sc = spark.sparkContext

FEATURES_NUM = 10000

### Данные

In [3]:
# Загрузка
df = (
    spark
    .read
    .format('json')
    .load('/labs/slaba02/DO_record_per_line.json')
)

df = df.cache()
df.is_cached

In [5]:
df.show(3)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
+--------------------+--------------------+---+----+--------------------+--------------+
only showing top 3 rows



In [51]:
### Проверка данных

# df.count()  # 28153
# df.select('desc').show(3, truncate=False)
# df.groupBy('provider').count().show()
# df.summary().toPandas().head(10)

In [11]:
# Курсы для индивидуального задания
target_courses_lst = [
    [23126, u'en', u'Compass - powerful SASS library that makes your life easier'], 
    [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], 
    [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], 
    [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], 
    [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], 
    [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']
]

# target_courses_lst

In [61]:
df.filter(F.col('id') == 864).select('lang', 'name').show(truncate=False)

+----+------------------------+
|lang|name                    |
+----+------------------------+
|ru  |Математическая экономика|
+----+------------------------+



### Вычисление TFIDF

In [18]:
### Расчет TFIDF для колонки с описанием 'desc'

# Важно!
# TFIDF нужно считать для описаний курсов (desc). 
# При извлечении слов из описания словом считаем то, что состоит из 
# латинских или кириллических букв или цифр, знаки препинания и прочие символы не учитываются.

@F.pandas_udf(ArrayType(StringType()))
def get_tokens_list(text_series):
    """Преобразовать колонку строк в колонку со списками токенов (слов)."""
    regex = re.compile(u'[\w\d]{2,}', re.U)  # паттерн для поиска слов (токенов)
    tokens_series = text_series.str.lower().apply(lambda row: regex.findall(row) if row else [])
    return tokens_series

# Токенизация описаний "desc" в список отдельных слов
df_tokenized = df.withColumn('desc', get_tokens_list('desc'))

# Расчет TF с помощью HashingTF
hashing_tf = HashingTF(numFeatures=1000, inputCol="desc", outputCol="tf_features")
df_tf = hashing_tf.transform(df_tokenized)

# Расчет IDF с помощью IDF
idf = IDF(inputCol='tf_features', outputCol='tfidf_features')
df_tfidf = idf.fit(df_tf).transform(df_tf)
df_tfidf = df_tfidf.cache()

df_tfidf.select().show(1)

<>:11: SyntaxWarning: invalid escape sequence '\w'
<>:11: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_32672/279080505.py:11: SyntaxWarning: invalid escape sequence '\w'
  regex = re.compile(u'[\w\d]{2,}', re.U)  # паттерн для поиска слов (токенов)


+---+--------------------+
| id|      tfidf_features|
+---+--------------------+
|  4|(1000,[7,12,17,23...|
+---+--------------------+
only showing top 1 row



In [21]:
### Проверка

# df_tfidf.show(n=1, vertical=True, truncate=False)
# df_tfidf.select('id', 'tfidf_features').show(n=1, vertical=True, truncate=False)

In [22]:
df_tfidf.dtypes

[('cat', 'string'),
 ('desc', 'array<string>'),
 ('id', 'bigint'),
 ('lang', 'string'),
 ('name', 'string'),
 ('provider', 'string'),
 ('tf_features', 'vector'),
 ('tfidf_features', 'vector')]

In [86]:
v1 = df_tfidf.select('tfidf_features').take(1)[0]['tfidf_features'].toArray()
v2 = df_tfidf.select('tfidf_features').take(2)[1]['tfidf_features'].toArray()

sum([a * b for a, b in zip(v1, v1)])

580.7860089849563

In [38]:
# type(df.filter(F.col('id') == 200).take(1)[0]['tfidf_features'])
# df_tfidf.filter(F.col('id') == 200).take(1)[0]['tfidf_features'].values

### Определение схожести для заданных фильмов

In [47]:
# vec = df_tfidf.filter(F.col('id') == 12).take(1)[0]['tfidf_features']

In [68]:
import math

def get_top10_neibs(df, trg_id, limit_rows=10):
    """Отбор для курса с заданным ID топ-10 наиболее похожих курсов."""
    
    # Вектор TFIDF и язык для курса с заданным id
    trg_tfidf_vec = df.filter(F.col('id') == trg_id).take(1)[0]['tfidf_features']
    trg_lang = df.filter(F.col('id') == trg_id).take(1)[0]['lang']
    
    # UDF для расчета косинуса между вектором заданного фильма и остальными
    @F.udf(FloatType())
    def compute_cos(vec):
        dot_product = float(sum(a * b for a, b in zip(vec.toArray(), trg_tfidf_vec.toArray())))
        vec1_norm = math.sqrt(sum([x * x for x in vec.values]))
        vec2_norm = math.sqrt(sum([x * x for x in trg_tfidf_vec.values]))
        if vec1_norm * vec2_norm == 0:
            return 0
        cos_alpha = dot_product / (vec1_norm * vec2_norm)
        return float(cos_alpha)
    
    # Расчет косинусов для всех фильмов
    df_cos_counted = df.withColumn(
        'cos_alpha', 
        compute_cos('tfidf_features')
    )

    # Выбор самых релевантных курсов
    res_df = (
        df_cos_counted
        .filter((F.col('lang') == trg_lang) & (F.col('id') != trg_id))
        .orderBy(['cos_alpha', 'name', 'id'], ascending=[False, True, True])
        .select('id', 'name', 'cos_alpha')
        .limit(10)
    )

    print(f'\n10 самых похожих курсов на курс с ID {trg_id}')
    res_df.show(10, truncate=False)

    return [row['id'] for row in res_df.collect()]


# Проверка
print(get_top10_neibs(df_tfidf, 864))

In [58]:
out_data = {}

for trg_course in target_courses_lst:
    trg_id = trg_course[0]
    out_data[str(trg_id)] = get_top10_neibs(df_tfidf, trg_id)

+-----+------------------------------------------------------------+--------------+
|id   |name                                                        |scalar_product|
+-----+------------------------------------------------------------+--------------+
|12949|8 Steps To Automating Tumblr For Profit by Moo Jelly        |0.69426197    |
|12470|Tumblr Marketing To Grow Your Business: Get Sales and Profit|0.67581594    |
|13665|The Next Step with Sass and Compass by Lisa Catalano        |0.6423741     |
|23958|How To Make A Free Private Blog Network - SEO With Tumblr   |0.62662965    |
|14760|Foundation 4: Incorporating Sass and Compass                |0.6048081     |
|13782|Assembling Sass Part 2                                      |0.5850956     |
|6915 |Tumblr Tutorial: Learn How to Use Tumblr - Udemy            |0.47067448    |
|16308|Up and Running with Tumblr                                  |0.44213796    |
|7650 |Unethical Decision Making in Organizations                  |0.435874

+-----+----------------------------------------------------------------+--------------+
|id   |name                                                            |scalar_product|
+-----+----------------------------------------------------------------+--------------+
|21609|Preparing for the AP* Computer Science A Exam — Part 1          |1.0           |
|21616|Preparing for the AP* Chemistry Exam- Part 2                    |0.6073332     |
|21608|Preparing for the AP* Chemistry Exam- Part 1                    |0.5856453     |
|308  |Linear and Integer Programming                                  |0.5346924     |
|21630|Preparing for the AP* Computer Science A Exam — Level Up! Part 3|0.5327079     |
|21628|Preparing for the AP* Computer Science A Exam — Level Up! Part 4|0.5327079     |
|8069 |Programming Cloud Services for Android Handheld Systems         |0.52959216    |
|21623|Preparing for the AP* Computer Science A Exam — Level Up! Part 1|0.5238278     |
|81   |Programming Mobile Applic

+-----+-------------------------------------------------------------------------------+--------------+
|id   |name                                                                           |scalar_product|
+-----+-------------------------------------------------------------------------------+--------------+
|11431|Excel Básico by Alfonso Rinsche                                                |0.75317734    |
|5687 |Excel para principiantes by Nicolás Martínez                                   |0.6665305     |
|17964|Excel 2010 Súper Fácil - Nivel Intermedio by Profesor 24/7                     |0.6628488     |
|11575|Curso de Excel Avanzado en tres (3) horas by Koul Advanced Learning            |0.6620905     |
|12247|Aprende Excel fácil y rápido by Elio Reyes                                     |0.6556602     |
|17961|Excel 2010 - Básico by Prof. Lucía Miralles                                    |0.64242023    |
|16694|Excel 2010 Súper Fácil - Nivel Básico by Profesor 24/7            

+-----+-------------------------------------------------------------------------------------+--------------+
|id   |name                                                                                 |scalar_product|
+-----+-------------------------------------------------------------------------------------+--------------+
|16488|Aprendizaje basado en problemas by UNID Universidad Interamericana para el Desarrollo|0.67497635    |
|10447|Mejores calificaciones usando tu dispositivo móvil by Ph. D. Dante Ramírez           |0.59606934    |
|13461|TIC para la enseñanza by Julia García Escudero                                       |0.56841165    |
|7833 |Coaching en el contexto educativo by Julia García Escudero                           |0.55487025    |
|468  |Tecnologías de información y comunicación en la educación                            |0.5442968     |
|9048 |Aprender a leer a los 3: Método Doman en preescolar by Elisa Guerra                  |0.54181075    |
|9784 |Herramientas

In [62]:
{k: out_data[k] for k in sorted(out_data)}

{'11556': [16488, 10447, 13461, 7833, 468, 9048, 9784, 9289, 23357, 19330],
 '13702': [864, 804, 13057, 1218, 20596, 895, 984, 8313, 21079, 28074],
 '16627': [11431, 5687, 17964, 11575, 12247, 17961, 16694, 5558, 10738, 12660],
 '16704': [1250, 1247, 875, 8207, 1164, 1365, 1132, 18331, 1232, 22210],
 '21617': [21609, 21616, 21608, 308, 21630, 21628, 8069, 21623, 81, 156],
 '23126': [12949, 12470, 13665, 23958, 14760, 13782, 6915, 16308, 7650, 15909]}

In [59]:
out_data

{'23126': [12949, 12470, 13665, 23958, 14760, 13782, 6915, 16308, 7650, 15909],
 '21617': [21609, 21616, 21608, 308, 21630, 21628, 8069, 21623, 81, 156],
 '16627': [11431, 5687, 17964, 11575, 12247, 17961, 16694, 5558, 10738, 12660],
 '11556': [16488, 10447, 13461, 7833, 468, 9048, 9784, 9289, 23357, 19330],
 '16704': [1250, 1247, 875, 8207, 1164, 1365, 1132, 18331, 1232, 22210],
 '13702': [864, 804, 13057, 1218, 20596, 895, 984, 8313, 21079, 28074]}

### Сохранение результатов

In [65]:
import json

out_data = {k: out_data[k] for k in sorted(out_data)}

with open('lab02.json', 'w', encoding='utf8') as f_out:
    json.dump(out_data, f_out, indent=4)

### Завершение контекста

In [66]:
spark.stop()